# ДЗ_12 Text Summarization

### Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
Проверить насколько хорошо она суммаризирует текст.

2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [1]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00


In [2]:
!pip install rouge
!pip install razdel

In [3]:
import pandas as pd
import numpy as np

**Загрузим датасет.**

In [4]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [5]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [6]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

Посмотрим на содержимое датасета.

In [7]:
test_records[0]

{'url': 'https://www.gazeta.ru/science/2020/02/14_a_12960289.shtml',
 'text': 'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, к

In [8]:
test_records[0].keys()

dict_keys(['url', 'text', 'title', 'summary', 'date'])

In [9]:
print(min([record["date"] for record in train_records]), end=' - ')
print(max([record["date"] for record in train_records]))
print(min([record["date"] for record in val_records]), end=' - ')
print(max([record["date"] for record in val_records]))
print(min([record["date"] for record in test_records]), end=' - ')
print(max([record["date"] for record in test_records]))

2010-06-01 10:35:49 - 2019-05-31 23:56:26
2019-06-01 08:30:00 - 2019-09-30 23:11:23
2019-10-01 08:23:02 - 2020-03-23 22:16:23


В качестве метрик будем использовать BLEU и ROUGE.

In [10]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [11]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


In [12]:
import torch

**В качестве готовой модели возьмем 'cointegrated/rut5-base-absum'.**

In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, legacy=False)
# model.cuda();
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs
):

    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [15]:
sum_pred = summarize(test_records[1]['text'], len(test_records[1]['summary'].split()))

In [16]:
sum_pred

'В Бурятии прошел праздничный концерт «Танцуют все!» на телеканале «Россия»'

In [17]:
test_records[1]['summary']

'25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.'

In [18]:
df_test = pd.DataFrame(test_records)

In [19]:
df_test

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24
...,...,...,...,...,...
5765,https://www.gazeta.ru/culture/2019/11/01/a_127...,Стриминговый сервис Netflix на итальянском фес...,Новая «Игра престолов»? Netflix показал трейле...,Стриминговый сервис Netflix опубликовал трейле...,2019-11-01 17:10:22
5766,https://www.gazeta.ru/culture/2019/11/05/a_127...,Голливудская актриса Анджелина Джоли и звезда ...,«Это было ужасно»: Джоли эвакуировали со съемо...,Актеры Анджелина Джоли и Ричард Мэдден были эв...,2019-11-05 20:09:26
5767,https://www.gazeta.ru/science/2019/11/22_a_128...,Университет штата Огайо удалил пресс-релиз об ...,«Это камни»: ученого затравили за змей на Марсе,Университет штата Огайо удалил пресс-релиз о н...,2019-11-22 15:02:51
5768,https://www.gazeta.ru/science/2020/01/27_a_129...,Китайским коронавирусом могут быть заражены ок...,«До 200 тысяч зараженных»: как распространяетс...,Китайским коронавирусом могут быть заражены ок...,2020-01-27 13:47:36


Для ускорения процесса тестирования модели уменьшим размер датасета.

In [20]:
df_mini = df_test[:50]

In [21]:
df_mini['summary_pred'] = df_mini.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

<ipython-input-21-313178c2af38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini['summary_pred'] = df_mini.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [22]:
df_mini

,url,text,title,summary,date,summary_pred
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11,Американское аэрокосмическое агентство NASA об...
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13,В Бурятии прошел праздничный концерт «Танцуют ...
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08,В Белоруссии впервые прошли выборы членов сове...
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06,"Надежда Бабкина рассказала, как ей удалось сбр..."
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24,В Верховной Раде раскритиковали законопроект о...
5,https://www.gazeta.ru/army/2019/11/13/12810158...,Журналист Питер Суппли Бенсон датского издания...,«Новый кулак в Арктике»: в Дании испугались «И...,Датского журналиста обеспокоила активность Рос...,2019-11-13 13:03:37,Журналист Питер Суппли Бенсон назвал «новым ку...
6,https://www.gazeta.ru/politics/2019/10/03_a_12...,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,2019-10-03 20:46:08,На заседании Парламентской ассамблеи Совета Ев...
7,https://www.gazeta.ru/politics/2020/01/03_a_12...,Самые известные российские пранкеры Владимир «...,Именем Греты Тунберг: Вован и Лексус разыграли...,Пранкеры Вован и Лексус разыграли члена конгре...,2020-01-03 19:25:16,Российские пранкеры Владимир «Вован» Кузнецов ...
8,https://www.gazeta.ru/culture/2019/11/06/a_127...,Российский актер и театральный педагог Виталий...,Умер актер сериала «Улицы разбитых фонарей»,Актер сериала «Улицы разбитых фонарей» Виталий...,2019-11-06 14:17:33,Российский актер Виталий Жигалин ушел из жизни...
9,https://www.gazeta.ru/business/2020/01/23/1292...,Госдума одобрила в первом чтении внесенные пре...,Дождались: работающим пенсионерам восстановят ...,Работающие пенсионеры дождались индексации. Эт...,2020-01-23 19:32:12,Госдума одобрила в первом чтении поправки в Ко...


In [23]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in records.iterrows():
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

In [24]:
calc_lead_n_score(df_mini, n=10)

Count: 50
Ref: 75 лет назад полной неудачей завершилось антинацистское восстание в варшаве. силы повстанцев капитулировали и сдали оружие. немцы подвергли город катастрофическим разрушениям, угнав многих уцелевших варшавян на принудительные работы. плачевные итоги выступления до сих пор являются предметом острых дискуссий. одни считают истинным виновником провала красную армию, не поддержавшую усилия поляков. другие указывают на то, что повстанцы не согласовали свое выступление с иосифом сталиным, который мог бы помочь, но чуть позже.
Hyp: 2 октября 1944 года полной капитуляцией повстанцев окончилось знаменитое варшавское восстание – одно из крупнейших антинацистских выступлений второй мировой войны, в ходе которого борьба за власть между двумя лагерями движения сопротивления определила отсутствие единства и привела к итоговой неудаче. повстанцы имели отдельные успехи, однако, не обладая тяжелой артиллерией и авиацией, допуская тактические ошибки и демонстрируя разобщенность, их подраз

In [25]:
calc_lead_n_score(df_mini, summary_col = 'summary_pred', n=10)

Count: 50
Ref: в варшаве завершилось знаменитое варшавское восстание. по словам военачальников, военная карьера генерала бур-коморовского началась на нелегальном положении
Hyp: 2 октября 1944 года полной капитуляцией повстанцев окончилось знаменитое варшавское восстание – одно из крупнейших антинацистских выступлений второй мировой войны, в ходе которого борьба за власть между двумя лагерями движения сопротивления определила отсутствие единства и привела к итоговой неудаче. повстанцы имели отдельные успехи, однако, не обладая тяжелой артиллерией и авиацией, допуская тактические ошибки и демонстрируя разобщенность, их подразделения подверглись разгрому и были вынуждены сдаться. уцелеть в этой бойне посчастливилось немногим. активно поддержанное простым народом, но плохо скоординированное и, по оценкам историков, недостаточно продуманное восстание вспыхнуло в варшаве 1 августа 1944 года. организацию выступления взяло на себя базировавшееся в лондоне польское правительство в изгнании, опи

### ВЫВОД: выбранная нами предобученная модель уверенно справляется со своей задачей на данном датасете.